# Modelling NIQS Hardware

In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import sys
from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

#from src_tf_noBP import *
from src import *
np.set_printoptions(threshold=sys.maxsize)

## Strong Model Hypothesis (Same Low Rank as Target Map)

### GD

In [44]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]


np.random.seed(42)
X_target = generate_ginibre(d**2, 2)

choi_target = generate_choi(X_target)
print(np.trace(choi_target))

state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]

model1 = ModelQuantumMap(n = 3, 
                         rank = 2, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.5, 
                         h = 1e-4)

model1.train(num_iter = 1000, 
             use_adam = False)

(8-2.636237582398504e-16j)


0: 0.061, 0.259
1: 0.163, 0.509
2: 0.076, 0.437
3: 0.051, 0.430
4: 0.018, 0.483
5: 0.043, 0.352
6: 0.070, 0.361
7: 0.035, 0.487
8: 0.097, 0.349
9: 0.037, 0.482
10: 0.039, 0.327
11: 0.039, 0.421
12: 0.087, 0.374
13: 0.030, 0.372
14: 0.051, 0.361
15: 0.089, 0.422
16: 0.072, 0.235
17: 0.014, 0.378
18: 0.041, 0.330
19: 0.026, 0.399
20: 0.109, 0.490
21: 0.045, 0.315
22: 0.000, 0.294
23: 0.125, 0.586
24: 0.043, 0.500
25: 0.125, 0.552
26: 0.017, 0.347
27: 0.082, 0.441
28: 0.013, 0.208
29: 0.105, 0.347
30: 0.065, 0.403
31: 0.099, 0.459
32: 0.156, 0.282
33: 0.104, 0.407
34: 0.119, 0.578
35: 0.017, 0.262
36: 0.006, 0.436
37: 0.195, 0.608
38: 0.014, 0.435
39: 0.028, 0.425
40: 0.131, 0.299
41: 0.063, 0.415
42: 0.048, 0.489
43: 0.022, 0.373
44: 0.076, 0.360
45: 0.165, 0.600
46: 0.232, 0.614
47: 0.031, 0.351
48: 0.047, 0.327
49: 0.032, 0.393
50: 0.065, 0.381
51: 0.099, 0.504
52: 0.002, 0.415
53: 0.025, 0.513
54: 0.103, 0.508
55: 0.025, 0.274
56: 0.086, 0.427
57: 0.048, 0.377
58: 0.013, 0.463
59: 0.0

KeyboardInterrupt: 

### ADAM

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
X_target = generate_ginibre(d**2, 2)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model2 = ModelQuantumMap(n = 3, 
                         rank = 2, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model2.train(num_iter = 1000, 
             use_adam = True)

In [ ]:
fig=plt.figure(figsize=(6,4), dpi = 130, facecolor='w', edgecolor='k')
plt.plot(model1.fid_list)
plt.plot(model2.fid_list)
plt.xlabel("Iteration")
plt.ylabel("Fidelity")
plt.legend(["GD", "ADAM"])
plt.grid()
plt.show()

### Compare Target Quantum Map to Fitted Models with a Random Pure state

In [ ]:
for i in range(5):

    state_random = generate_state(d, 1)
    state_target = apply_map(state_random, choi_target)

    choi_model1 = generate_choi(model1.X_model)
    state_model1 = apply_map(state_random, choi_model1)
    print(state_fidelity(state_model1, state_target))

    choi_model2 = generate_choi(model2.X_model)
    state_model2 = apply_map(state_random, choi_model2)
    print(state_fidelity(state_model2, state_target))
    print("----")

## More Testing

### Full Rank Model (No Target Hypothesis)

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
X_target = generate_ginibre(d**2, 2)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]

model3 = ModelQuantumMap(n = 3, 
                         rank = d**2, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model3.train(num_iter = 1000, 
             use_adam = True)

### Over-Constrained Model

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
X_target = generate_ginibre(d**2, 4)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]

model4 = ModelQuantumMap(n = 3, 
                         rank = 2, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model4.train(num_iter = 1000, 
             use_adam = True)

### Full Rank Target

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
X_target = generate_ginibre(d**2, d**2)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model5 = ModelQuantumMap(n = 3, 
                         rank = d**2, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model5.train(num_iter = 1000, 
             use_adam = True)

In [ ]:
np.mean((choi_target - choi_model1)**2)

fig=plt.figure(figsize=(6,4), dpi = 130, facecolor='w', edgecolor='k')
plt.plot(model3.fid_list)
plt.plot(model4.fid_list)
plt.plot(model5.fid_list)
plt.xlabel("Iteration")
plt.ylabel("Fidelity")
plt.legend(["Full Rank Model", "Over-Contrained", "Full Rank Target"])
plt.grid()
plt.show()

### Four Qubits

In [ ]:
n = 4
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
X_target = generate_ginibre(d**2, 6)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model6 = ModelQuantumMap(n = 4, 
                         rank = 6, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model6.train(num_iter = 2000, 
             use_adam = True)

## Constrained Inputs

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]
random.shuffle(state_input_list)
state_input_list = state_input_list[:100]


np.random.seed(42)
X_target = generate_ginibre(d**2, 4)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model6 = ModelQuantumMap(n = 3, 
                         rank = 4, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model6.train(num_iter = 1000, 
             use_adam = True)

In [ ]:
for i in range(5):

    state_random = generate_state(d, 1)
    state_target = apply_map(state_random, choi_target)

    choi_model6 = generate_choi(model6.X_model)
    state_model6 = apply_map(state_random, choi_model6)
    print(state_fidelity(state_model6, state_target))
    print("----")

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]
random.shuffle(state_input_list)
state_input_list = state_input_list[:50]


np.random.seed(42)
X_target = generate_ginibre(d**2, 4)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model6 = ModelQuantumMap(n = 3, 
                         rank = 4, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model6.train(num_iter = 1000, 
             use_adam = True)

In [ ]:
for i in range(5):

    state_random = generate_state(d, 1)
    state_target = apply_map(state_random, choi_target)

    choi_model6 = generate_choi(model6.X_model)
    state_model6 = apply_map(state_random, choi_model6)
    print(state_fidelity(state_model6, state_target))
    print("----")

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]
random.shuffle(state_input_list)
state_input_list = state_input_list[:25]


np.random.seed(42)
X_target = generate_ginibre(d**2, 4)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model6 = ModelQuantumMap(n = 3, 
                         rank = 4, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model6.train(num_iter = 1000, 
             use_adam = True)

In [ ]:
for i in range(5):

    state_random = generate_state(d, 1)
    state_target = apply_map(state_random, choi_target)

    choi_model6 = generate_choi(model6.X_model)
    state_model6 = apply_map(state_random, choi_model6)
    print(state_fidelity(state_model6, state_target))
    print("----")

In [ ]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]
random.shuffle(state_input_list)
state_input_list = state_input_list[:10]


np.random.seed(42)
X_target = generate_ginibre(d**2, 4)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model6 = ModelQuantumMap(n = 3, 
                         rank = 4, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model6.train(num_iter = 1000, 
             use_adam = True)

In [ ]:
for i in range(5):

    state_random = generate_state(d, 1)
    state_target = apply_map(state_random, choi_target)

    choi_model6 = generate_choi(model6.X_model)
    state_model6 = apply_map(state_random, choi_model6)
    print(state_fidelity(state_model6, state_target))
    print("----")

## 4 Qubits

In [ ]:
n = 4
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]
random.shuffle(state_input_list)
state_input_list = state_input_list[:100]


np.random.seed(42)
X_target = generate_ginibre(d**2, 4)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


model6 = ModelQuantumMap(n = 4, 
                         rank = 4, 
                         state_input_list = state_input_list,
                         state_target_list = state_target_list,
                         lr = 0.05, 
                         h = 1e-4)

model6.train(num_iter = 1000, 
             use_adam = True)

In [ ]:
for i in range(5):

    state_random = generate_state(d, 1)
    state_target = apply_map(state_random, choi_target)

    choi_model6 = generate_choi(model6.X_model)
    state_model6 = apply_map(state_random, choi_model6)
    print(state_fidelity(state_model6, state_target))
    print("----")

## Pauli String Data

In [56]:
n = 2
d = 2**n
rank = 2
num_samples = 10000

np.random.seed(42)
state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

X_target = generate_ginibre(d**2, rank)
choi_target = generate_choi(X_target)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]

data = np.zeros((6**n, 6**n, d))

for i in tqdm(range(num_samples)):   
    idx1 = np.random.randint(6**n)
    state = state_target_list[idx1]
    idx2 = np.random.randint(6**n)
    basis = prepare_input(numberToBase(idx2, 6, n), return_unitary = True)
    idx3 = sample_bitstring(state, basis, return_index = True)
    data[idx1, idx2, idx3] += 1

In [57]:
print(data)

[[[ 3.  0.  5.  0.]
  [ 3.  1.  0.  1.]
  [ 8.  3.  2.  1.]
  [ 6.  0.  0.  4.]
  [ 4.  1.  4.  2.]
  [ 6.  0.  0.  0.]
  [ 1.  2.  3.  5.]
  [ 3.  4.  1.  2.]
  [ 0.  5.  1.  1.]
  [ 0.  3.  2.  1.]
  [ 0.  6.  0.  1.]
  [ 1.  4.  1.  0.]
  [ 1.  1.  1.  0.]
  [ 1.  1.  0.  0.]
  [ 3.  2.  0.  0.]
  [ 1.  3.  1.  0.]
  [ 4.  1.  1.  1.]
  [ 1.  0.  0.  0.]
  [ 1.  4.  6.  3.]
  [ 4.  1.  0.  1.]
  [ 3.  1.  0.  2.]
  [ 0.  1.  0.  3.]
  [ 2.  2.  0.  2.]
  [ 2.  1.  0.  1.]
  [ 1.  1.  3.  3.]
  [ 4.  1.  0.  4.]
  [ 1.  1.  2.  1.]
  [ 3.  1.  0.  0.]
  [ 2.  2.  3.  3.]
  [ 4.  2.  1.  3.]
  [ 0.  1.  1.  1.]
  [ 6.  3.  0.  2.]
  [ 3.  1.  1.  1.]
  [ 5.  0.  2.  2.]
  [ 8.  0.  0.  2.]
  [ 4.  4.  1.  3.]]

 [[ 4.  2.  0.  0.]
  [ 1.  1.  2.  5.]
  [ 0.  3.  4.  1.]
  [ 4.  6.  2.  0.]
  [ 3.  3.  3.  0.]
  [ 1.  2.  2.  0.]
  [ 2.  2.  2.  0.]
  [ 1.  0.  3.  2.]
  [ 4.  3.  1.  1.]
  [ 3.  3.  1.  1.]
  [ 0.  1.  2.  1.]
  [ 3.  2.  0.  4.]
  [ 3.  3.  2.  1.]
  [ 2.  0.  4.  2.

## Kraus Form

In [8]:
n = 2
d = 2**n
rank = 2

np.random.seed(42)
state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]
random.shuffle(state_input_list)
state_input_list = state_input_list


np.random.seed(42)
X_target = generate_ginibre(rank*d, d)
kraus_target = generate_kraus(X_target, d, rank)
state_target_list = [apply_kraus(state_input, kraus_target) for state_input in state_input_list]

In [9]:
np.trace(state_target_list[0])

(0.9999999999999997-1.0408340855860843e-17j)

In [10]:
state_target_list[0]

array([[ 0.37493162-1.38777878e-17j, -0.10326568+1.96998942e-01j,
         0.10811999-1.97533638e-01j, -0.08491656-1.74851862e-01j],
       [-0.10326568-1.96998942e-01j,  0.18991841+6.93889390e-18j,
        -0.18852286+2.53149599e-03j, -0.11038447+1.74890631e-01j],
       [ 0.10811999+1.97533638e-01j, -0.18852286-2.53149599e-03j,
         0.18776768-1.04083409e-17j,  0.11434656-1.69439786e-01j],
       [-0.08491656+1.74851862e-01j, -0.11038447-1.74890631e-01j,
         0.11434656+1.69439786e-01j,  0.24738229+6.93889390e-18j]])